In [57]:
import xml.dom.minidom

mapping = {}
dom = xml.dom.minidom.parse('eastenders.collection.xml')
video_file_list = dom.getElementsByTagName('VideoFileList')[0]
for video_file in video_file_list.getElementsByTagName('VideoFile'):
    video_id = int(video_file.getElementsByTagName('id')[0].childNodes[0].data)
    uri = video_file.getElementsByTagName('filename')[0].childNodes[0].data[:-4]
    mapping[video_id] = uri

In [58]:
def to_seconds(t):
    hours, minutes, seconds, frames = t[1:-3].split(':')
    hours = int(hours)
    minutes = int(minutes)
    seconds = int(seconds) + int(frames) / 25
    return 3600 * hours + 60 * minutes + seconds

In [61]:
from pandas import read_table
names = ['video_id', 'shot_id', 'start', 'stop']
converters = {'start': to_seconds, 
              'stop': lambda t: to_seconds(t) + 1/25}
data = read_table('eastenders.masterShotReferenceTable', delim_whitespace=True, 
                  names=names, converters=converters, index_col='shot_id')
groups = data.groupby('video_id')

In [72]:
# dev
shots = groups.get_group(0)
with open('../Eastenders/data/shots.dev.txt', 'w') as fp:
    uri = mapping[0]
    for shot_id, shot in shots.iterrows():
        fp.write(f'{uri} {shot.start:.3f} {shot.stop:.3f} {shot_id}\n')

In [73]:
# tst
with open('../Eastenders/data/shots.tst.txt', 'w') as fp:
    for video_id, shots in groups:
        if video_id == 0:
            continue
        uri = mapping[video_id]
        for shot_id, shot in shots.iterrows():
            fp.write(f'{uri} {shot.start:.3f} {shot.stop:.3f} {shot_id}\n')

In [80]:
!wc -l ../Eastenders/data/shots.dev.txt
!gzip ../Eastenders/data/shots.dev.txt

    1987 ../Eastenders/data/shots.dev.txt


In [81]:
!wc -l ../Eastenders/data/shots.tst.txt
!gzip ../Eastenders/data/shots.tst.txt

  469539 ../Eastenders/data/shots.tst.txt


In [82]:
!ls ../Eastenders/data

eastenders.all.lst   shots.tst.txt.gz     subtitles.tst.txt.gz
shots.dev.txt.gz     subtitles.dev.txt.gz
